<a href="https://colab.research.google.com/github/aryan-kazuha/ML-DL_projects/blob/main/LSTMvsGRU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [21]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

import os
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense,LSTM,GRU,Dropout,BatchNormalization
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping,ReduceLROnPlateau

import yfinance as yf
from sklearn.preprocessing import MinMaxScaler


In [22]:
aapl = yf.download("AAPL",start='2018-01-01',end = '2024-12-31')
df = aapl.reset_index()

[*********************100%***********************]  1 of 1 completed


In [23]:
df.columns

MultiIndex([(  'Date',     ''),
            ( 'Close', 'AAPL'),
            (  'High', 'AAPL'),
            (   'Low', 'AAPL'),
            (  'Open', 'AAPL'),
            ('Volume', 'AAPL')],
           names=['Price', 'Ticker'])

In [24]:
df.columns = ["Date","Close",'High','Low','Open','Volume']

In [25]:
features = df.columns.tolist()
features.remove('Date')

In [26]:
def linePlot(df,features):
  n_rows = int(np.ceil(len(features)//2))
  n_cols = 2
  for idx,feature in enumerate(features):
    plot = px.line(df,x="Date",y = feature,title = f"Aapl {feature} Price")
    plt.tight_layout()
    plot.show()


In [27]:
linePlot(df,features)

<Figure size 1200x800 with 0 Axes>

In [28]:
df.head()

,Date,Close,High,Low,Open,Volume
0,2018-01-02,40.426830,40.436220,39.722775,39.933994,102223600
1,2018-01-03,40.419792,40.964263,40.356430,40.490198,118071600
2,2018-01-04,40.607536,40.710798,40.384586,40.492539,89738400
3,2018-01-05,41.069855,41.156687,40.612220,40.703747,94640000
4,2018-01-08,40.917313,41.213014,40.818742,40.917313,82271200


In [29]:
small_df = df[["Date","Close"]].copy()

In [30]:
scaler = MinMaxScaler()
scaled_data_mul = scaler.fit_transform(df[features])
scaled_data_uni = scaler.fit_transform(df[["Close"]])

In [31]:
def create_sequences(data, seq_length=60):
    X, y = [], []
    for i in range(seq_length, len(data)):
        X.append(data[i-seq_length:i])
        y.append(data[i])
    return np.array(X), np.array(y)

SEQ_LEN = 60
X_mul, y_mul = create_sequences(scaled_data_mul, SEQ_LEN)
X_uni, y_uni = create_sequences(scaled_data_uni, SEQ_LEN)


split = int(0.8 * len(X_mul))
X_train_mul, X_test_mul = X_mul[:split], X_mul[split:]
y_train_mul, y_test_mul = y_mul[:split], y_mul[split:]

X_train_uni, X_test_uni = X_uni[:split], X_uni[split:]
y_train_uni, y_test_uni = y_uni[:split], y_uni[split:]

In [32]:
optimizer = Adam(learning_rate=1e-4)
callbacks = [EarlyStopping(monitor = "val_loss",patience=10,restore_best_weights=True),ReduceLROnPlateau(monitor = "val_loss",patience = 5,factor = 0.5)]

In [45]:
optimizer_gru = Adam(learning_rate=1e-3)
callbacks_gru = [EarlyStopping(monitor = "val_loss",patience=10,restore_best_weights=True),ReduceLROnPlateau(monitor = "val_loss",patience = 5,factor = 0.5)]

In [33]:
def lstm(input_shape):
  model = Sequential()
  model.add(LSTM(128,input_shape = input_shape,return_sequences = False))
  model.add(BatchNormalization())
  model.add(Dropout(0.3))
  model.add(Dense(32,activation = "relu"))
  model.add(Dropout(0.3))
  model.add(Dense(1))

  model.compile(
      optimizer = optimizer,
      loss ='mse'
  )
  return model

In [48]:
def gru(input_shape):
  model = Sequential()
  model.add(GRU(128,input_shape = input_shape,return_sequences = False))
  model.add(BatchNormalization())
  model.add(Dropout(0.3))
  model.add(Dense(32,activation = "relu"))
  model.add(Dropout(0.3))
  model.add(Dense(1))

  model.compile(
      optimizer = optimizer_gru,
      loss = 'mse'
  )
  return model

In [35]:
Lstm_uni = lstm(X_train_uni.shape[1:])
Lstm_mul = lstm(X_train_mul.shape[1:])

/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning:

Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.



In [42]:
Gru_uni = gru(X_train_uni.shape[1:])
Gru_mul = gru(X_train_mul.shape[1:])

/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning:

Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.



In [36]:
tf.config.run_functions_eagerly(True)

In [37]:
history_lstm_uni = Lstm_uni.fit(X_train_uni,y_train_uni,epochs = 50,validation_data = (X_test_uni,y_test_uni),callbacks=callbacks)

Epoch 1/50
 1/43 ━━━━━━━━━━━━━━━━━━━━ 5s 131ms/step - loss: 0.3814

/usr/local/lib/python3.11/dist-packages/tensorflow/python/data/ops/structured_function.py:258: UserWarning:

Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.



43/43 ━━━━━━━━━━━━━━━━━━━━ 4s 92ms/step - loss: 0.2400 - val_loss: 0.4437 - learning_rate: 1.0000e-04
Epoch 2/50
43/43 ━━━━━━━━━━━━━━━━━━━━ 5s 105ms/step - loss: 0.1340 - val_loss: 0.4373 - learning_rate: 1.0000e-04
Epoch 3/50
43/43 ━━━━━━━━━━━━━━━━━━━━ 4s 90ms/step - loss: 0.1079 - val_loss: 0.4031 - learning_rate: 1.0000e-04
Epoch 4/50
43/43 ━━━━━━━━━━━━━━━━━━━━ 6s 107ms/step - loss: 0.0979 - val_loss: 0.3390 - learning_rate: 1.0000e-04
Epoch 5/50
43/43 ━━━━━━━━━━━━━━━━━━━━ 4s 87ms/step - loss: 0.0759 - val_loss: 0.2979 - learning_rate: 1.0000e-04
Epoch 6/50
43/43 ━━━━━━━━━━━━━━━━━━━━ 5s 90ms/step - loss: 0.0718 - val_loss: 0.2919 - learning_rate: 1.0000e-04
Epoch 7/50
43/43 ━━━━━━━━━━━━━━━━━━━━ 6s 101ms/step - loss: 0.0677 - val_loss: 0.2476 - learning_rate: 1.0000e-04
Epoch 8/50
43/43 ━━━━━━━━━━━━━━━━━━━━ 5s 89ms/step - loss: 0.0559 - val_loss: 0.2158 - learning_rate: 1.0000e-04
Epoch 9/50
43/43 ━━━━━━━━━━━━━━━━━━━━ 6s 103ms/step - loss: 0.0520 - val_loss: 0.1883 - learning_rate: 1

In [38]:
history_lstm_mul = Lstm_uni.fit(X_train_mul,y_train_mul,epochs = 50,validation_data = (X_test_mul,y_test_mul),callbacks=callbacks)

Epoch 1/50
43/43 ━━━━━━━━━━━━━━━━━━━━ 4s 91ms/step - loss: 0.0597 - val_loss: 75.1553 - learning_rate: 2.5000e-05
Epoch 2/50
43/43 ━━━━━━━━━━━━━━━━━━━━ 6s 105ms/step - loss: 0.1450 - val_loss: 121.0525 - learning_rate: 2.5000e-05
Epoch 3/50
43/43 ━━━━━━━━━━━━━━━━━━━━ 4s 88ms/step - loss: 0.0886 - val_loss: 124.8383 - learning_rate: 2.5000e-05
Epoch 4/50
43/43 ━━━━━━━━━━━━━━━━━━━━ 6s 98ms/step - loss: 0.0836 - val_loss: 104.6485 - learning_rate: 2.5000e-05
Epoch 5/50
43/43 ━━━━━━━━━━━━━━━━━━━━ 4s 98ms/step - loss: 0.0787 - val_loss: 91.2114 - learning_rate: 2.5000e-05
Epoch 6/50
43/43 ━━━━━━━━━━━━━━━━━━━━ 4s 89ms/step - loss: 0.0751 - val_loss: 75.5250 - learning_rate: 2.5000e-05
Epoch 7/50
43/43 ━━━━━━━━━━━━━━━━━━━━ 6s 107ms/step - loss: 0.0669 - val_loss: 71.2316 - learning_rate: 1.2500e-05
Epoch 8/50
43/43 ━━━━━━━━━━━━━━━━━━━━ 4s 90ms/step - loss: 0.0703 - val_loss: 67.5985 - learning_rate: 1.2500e-05
Epoch 9/50
43/43 ━━━━━━━━━━━━━━━━━━━━ 4s 89ms/step - loss: 0.0635 - val_loss: 66.16

In [43]:
history_gru_uni = Gru_uni.fit(X_train_uni,y_train_uni,epochs = 50,validation_data = (X_test_uni,y_test_uni),callbacks=callbacks)

Epoch 1/50


/usr/local/lib/python3.11/dist-packages/tensorflow/python/data/ops/structured_function.py:258: UserWarning:

Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.



43/43 ━━━━━━━━━━━━━━━━━━━━ 5s 103ms/step - loss: 0.1738 - val_loss: 0.3753 - learning_rate: 0.0010
Epoch 2/50
43/43 ━━━━━━━━━━━━━━━━━━━━ 4s 88ms/step - loss: 0.0494 - val_loss: 0.3063 - learning_rate: 0.0010
Epoch 3/50
43/43 ━━━━━━━━━━━━━━━━━━━━ 6s 99ms/step - loss: 0.0372 - val_loss: 0.2721 - learning_rate: 0.0010
Epoch 4/50
43/43 ━━━━━━━━━━━━━━━━━━━━ 5s 89ms/step - loss: 0.0234 - val_loss: 0.2526 - learning_rate: 0.0010
Epoch 5/50
43/43 ━━━━━━━━━━━━━━━━━━━━ 4s 87ms/step - loss: 0.0185 - val_loss: 0.2057 - learning_rate: 0.0010
Epoch 6/50
43/43 ━━━━━━━━━━━━━━━━━━━━ 8s 160ms/step - loss: 0.0114 - val_loss: 0.2005 - learning_rate: 0.0010
Epoch 7/50
43/43 ━━━━━━━━━━━━━━━━━━━━ 9s 123ms/step - loss: 0.0122 - val_loss: 0.2074 - learning_rate: 0.0010
Epoch 8/50
43/43 ━━━━━━━━━━━━━━━━━━━━ 9s 102ms/step - loss: 0.0109 - val_loss: 0.1512 - learning_rate: 0.0010
Epoch 9/50
43/43 ━━━━━━━━━━━━━━━━━━━━ 5s 114ms/step - loss: 0.0092 - val_loss: 0.1765 - learning_rate: 0.0010
Epoch 10/50
43/43 ━━━━━━━

In [51]:
loss_lstm_uni = history_lstm_uni.history['loss'][-1]
val_loss_lstm_uni = history_lstm_uni.history['val_loss'][-1]

loss_lstm_mul = history_lstm_mul.history['loss'][-1]
val_loss_lstm_mul = history_lstm_mul.history['val_loss'][-1]

loss_gru_uni = history_gru_uni.history['loss'][-1]
val_loss_gru_uni = history_gru_uni.history['val_loss'][-1]

data = {'Model': ['LSTM Unvariate', 'LSTM Multivariate', 'GRU Unvariate'],
        'Train Loss (MSE)': [loss_lstm_uni, loss_lstm_mul, loss_gru_uni],
        'Validation Loss (MSE)': [val_loss_lstm_uni, val_loss_lstm_mul, val_loss_gru_uni]}

df_results = pd.DataFrame(data)

df_results


,Model,Train Loss (MSE),Validation Loss (MSE)
0,LSTM Unvariate,0.020915,0.001973
1,LSTM Multivariate,0.043975,39.082069
2,GRU Unvariate,0.003559,0.002959


In [50]:
Lstm_uni.save("lstm_model.keras")
Lstm_mul.save("lstm_model_mul.keras")
Gru_uni.save("gru_model.keras")